In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.utils import shuffle


In [ ]:
class Graph():
    def __init__(self,max_hop=1,strategy="spatial"):
        self.max_hop = max_hop
        self.get_edge()
        self.hop_dis = self.get_hop_distance()
        self.get_adj(strategy)
    def __str__(self):
        return str(self.A)
    
    def get_edge(self):
        self.n_nodes = 20
        edges = [(4,3),(3,5),(5,6),(6,7),(7,8),(3,9),(9,10),
                 (10,11),(11,12),(3,2),(2,1),(1,13),(1,17),
                 (13,14),(14,15),(15,16),(17,18),(18,19),
                 (19,20)] + [(i,i) for i in range(self.n_nodes)]
        self.adjacency = np.zeros((self.n_nodes,self.n_nodes))
        for i, j in edges:
            self.adjacency[i-1,j-1] = 1
            self.adjacency[j-1,i-1] = 1
        self.center=2
    
    def get_adj(self,strategy):
        valid_hop = range(0,self.max_hop+1)
        adjacency = np.zeros((self.n_nodes,self.n_nodes))
        for hop in valid_hop:
            adjacency[self.hop_dis==hop]=1
        normalized_adjacency = self.get_normalized_g(adjacency)
        
        if strategy=='uniform':
            A = np.zeros((1,self.n_nodes,self.n_nodes))
            A[0] = normalized_adjacency
            self.A=A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.n_nodes,self.n_nodes))
                a_close = np.zeros((self.n_nodes,self.n_nodes))
                a_further = np.zeros((self.n_nodes,self.n_nodes))
                for i in range(self.n_nodes):
                    for j in range(self.n_nodes):
                        if self.hop_dis[j,i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[
                                    i, self.center]:
                                a_root[j, i] = normalized_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalized_adjacency[j, i]
                            else:
                                a_further[j, i] = normalized_adjacency[j, i]
                if hop==0:
                    A.append(a_root)
                else:
                    A.append(a_root+a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A=A
        else:
            raise ValueError("Do no t Exist This Method")
            
            
    def get_normalized_g(self,A):
        degrees = np.sum(self.adjacency,0)
        D = np.zeros((self.n_nodes,self.n_nodes))
        for i in range(self.n_nodes):
            if degrees[i] > 0:
                D[i,i] = degrees[i]**(-1)
        return np.dot(A,D)
        
    def get_hop_distance(self):
        A = self.adjacency
        hop_dis = np.zeros((self.n_nodes,self.n_nodes))+np.inf
        transfer_mat = [np.linalg.matrix_power(A,d) for d in range(self.max_hop+1)]
        arrival_mat = (np.stack(transfer_mat)>0)
        for d in range(self.max_hop,-1,-1):
            hop_dis[arrival_mat[d]] = d
        return hop_dis

In [ ]:
class ConvTemporalGraphical(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 t_kernel_size=1,
                 t_stride=1,
                 t_padding=0,
                 t_dilation=1,
                 bias=True):
        super().__init__()

        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(
            in_channels,
            out_channels * kernel_size,
            kernel_size=(t_kernel_size, 1),
            padding=(t_padding, 0),
            stride=(t_stride, 1),
            dilation=(t_dilation, 1),
            bias=bias)

    def forward(self, x, A):
        assert A.size(0) == self.kernel_size
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))
        return x.contiguous(), A

In [ ]:
class Model(nn.Module):
    """Spatial temporal graph convolutional networks.
    Args:
        in_channels (int): Number of channels in the input data
        num_class (int): Number of classes for the classification task
        graph_args (dict): The arguments for building the graph
        edge_importance_weighting (bool): If ``True``, adds a learnable
            importance weighting to the edges of the graph
        **kwargs (optional): Other parameters for graph convolution units
    Shape:
        - Input: :math:`(N, in_channels, T_{in}, V_{in}, M_{in})`
        - Output: :math:`(N, num_class)` where
            :math:`N` is a batch size,
            :math:`T_{in}` is a length of input sequence,
            :math:`V_{in}` is the number of graph nodes,
            :math:`M_{in}` is the number of instance in a frame.
    """

    def __init__(self, in_channels, num_class,
                 edge_importance_weighting=None, **kwargs):
        super().__init__()

        # load graph
        self.graph = Graph()
        A = torch.tensor(self.graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)

        # build networks
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 3
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        kwargs0 = {k: v for k, v in kwargs.items() if k != 'dropout'}
        self.st_gcn_networks = nn.ModuleList((
            st_gcn(in_channels, 32, kernel_size, 1, residual=False, **kwargs0),
            st_gcn(32, 32, kernel_size, 1, **kwargs),
            st_gcn(32, 32, kernel_size, 1, **kwargs),
            st_gcn(32, 32, kernel_size, 1, **kwargs),
            st_gcn(32, 64, kernel_size, 2, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 128, kernel_size, 2, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
        ))

        # initialize parameters for edge importance weighting
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(self.A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)

        # fcn for prediction
        self.fcn = nn.Conv2d(128, num_class, kernel_size=1)

    def forward(self, x):

        # data normalization
        N, C, V, T= x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)        
        # forwad
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x, _ = gcn(x, self.A * importance)

        # global pooling
        
        x = F.avg_pool2d(x, x.size()[2:])

        # prediction
        x = self.fcn(x)
        x = x.view(x.size(0), -1)

        return x

    def extract_feature(self, x):

        # data normalization
        N, C, V, T= x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)  

        # forwad
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x, _ = gcn(x, self.A * importance)

        _, c, t, v = x.size()
        feature = x.view(N, c, t, v).permute(0, 1, 2, 3)

        # prediction
        x = self.fcn(x)
        output = x.view(N, -1, t, v).permute(0, 1, 2, 3)

        return output, feature

class st_gcn(nn.Module):
    """Applies a spatial temporal graph convolution over an input graph sequence.
    Args:
        in_channels (int): Number of channels in the input sequence data
        out_channels (int): Number of channels produced by the convolution
        kernel_size (tuple): Size of the temporal convolving kernel and graph convolving kernel
        stride (int, optional): Stride of the temporal convolution. Default: 1
        dropout (int, optional): Dropout rate of the final output. Default: 0
        residual (bool, optional): If ``True``, applies a residual mechanism. Default: ``True``
    Shape:
        - Input[0]: Input graph sequence in :math:`(N, in_channels, T_{in}, V)` format
        - Input[1]: Input graph adjacency matrix in :math:`(K, V, V)` format
        - Output[0]: Outpu graph sequence in :math:`(N, out_channels, T_{out}, V)` format
        - Output[1]: Graph adjacency matrix for output data in :math:`(K, V, V)` format
        where
            :math:`N` is a batch size,
            :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
            :math:`T_{in}/T_{out}` is a length of input/output sequence,
            :math:`V` is the number of graph nodes.
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dropout=0,
                 residual=True):
        super().__init__()

        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        padding = ((kernel_size[0] - 1) // 2, 0)

        self.gcn = ConvTemporalGraphical(in_channels, out_channels,
                                         kernel_size[1])

        self.tcn = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                out_channels,
                out_channels,
                (kernel_size[0], 1),
                (stride, 1),
                padding,
            ),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(dropout, inplace=True),
        )

        if not residual:
            self.residual = lambda x: 0

        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x

        else:
            self.residual = nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size=1,
                    stride=(stride, 1)),
                nn.BatchNorm2d(out_channels),
            )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, A):

        res = self.residual(x)
        x, A = self.gcn(x, A)
        x = self.tcn(x) + res
        
        return self.relu(x), A

In [ ]:
class GCNDataset(Dataset):
    def __init__(self,filename,hasLabel=True):
        self.df = pd.read_csv(filename,header=None)
        if hasLabel:
            self.df['freq']=1./self.df.groupby(961)[961].transform('count')
            self.df = self.df.sample(frac=1,weights=self.df.freq).reset_index(drop=True)
        self.length = len(self.df)
        if hasLabel:
            self.X = torch.tensor(self.df.iloc[:,1:-2].values.astype('float32'))
            # N, C, V, T
            self.X = self.X.reshape((self.length ,16, 20, 3)).permute(0,3,2,1).contiguous()
            labels = self.df.iloc[:,-2].values.astype('int32')-1
            self.Y = np.zeros((labels.size, labels.max()+1))
            self.Y[np.arange(labels.size),labels] = 1
        else:
            self.X = torch.tensor(self.df.iloc[:,1:].values.astype('float32'))
            self.X = self.X.reshape((self.length ,16, 20, 3)).permute(0,3,2,1).contiguous()
            self.Y = np.zeros((self.length,49))
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        x = self.X[index]
        y = self.Y[index]
        return x, y 

In [ ]:
train_set = GCNDataset(filename="drive/MyDrive/SML/training_set.csv")
val_set = GCNDataset(filename="drive/MyDrive/SML/val_set.csv")
test_set = GCNDataset(filename="drive/MyDrive/SML/test.csv",hasLabel=False)
train_loader = DataLoader(train_set,batch_size=64)
val_loader = DataLoader(val_set,batch_size=64)
test_loader = DataLoader(test_set,batch_size=64)
net = Model(3,49)
#net.load_state_dict(torch.load("stgcn-best.pkl"))
gpu = 0 #gpu ID
net.cuda(gpu)
print()

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)
#opti.load_state_dict(torch.load('stgcn_best_optim.pkl'))
def accuracy(logit,target):
    a=(torch.argmax(logit,dim=1)==torch.argmax(target,dim=1)).sum()
    return a

def evaluate(model, criterion, dataloader, gpu):
    model.eval()
    acc = 0
    count = 0
    with torch.no_grad():
        for i,(x,y) in enumerate(dataloader):
            x,y = x.cuda(gpu), y.cuda(gpu)
            logits = model(x)
            acc+= accuracy(logits, y)
            count += 64

    return acc / count

def train():
    best_acc=0.35
    best_epoch = 0
    for epoch in range(200):
        print("epoch:",epoch)
        total = 0
        correct = 0
        for i, (x,y) in enumerate(train_loader):
            opti.zero_grad()
            x,y=x.cuda(gpu),y.cuda(gpu)
            opti.zero_grad()
            logit = net(x)
            loss = criterion(logit,y)
            loss.backward()
            opti.step()
            correct+=accuracy(logit,y)
            total+=64
        print("train acc:",correct/total)

        dev_acc = evaluate(net, criterion, val_loader, gpu)
        if dev_acc>best_acc:
            best_acc=dev_acc
            torch.save(net.state_dict(), 'stgcn-best.pkl')
            torch.save(opti.state_dict(), "stgcn_best_optim.pkl")
        print("dev_acc:",dev_acc)
train()

epoch: 0
train acc: tensor(0.0070, device='cuda:0')
dev_acc: tensor(0.0094, device='cuda:0')
epoch: 1
train acc: tensor(0.1225, device='cuda:0')
dev_acc: tensor(0.1646, device='cuda:0')
epoch: 2
train acc: tensor(0.1745, device='cuda:0')
dev_acc: tensor(0.1589, device='cuda:0')
epoch: 3
train acc: tensor(0.1886, device='cuda:0')
dev_acc: tensor(0.1630, device='cuda:0')
epoch: 4
train acc: tensor(0.2033, device='cuda:0')
dev_acc: tensor(0.1781, device='cuda:0')
epoch: 5
train acc: tensor(0.2154, device='cuda:0')
dev_acc: tensor(0.1880, device='cuda:0')
epoch: 6
train acc: tensor(0.2230, device='cuda:0')
dev_acc: tensor(0.1948, device='cuda:0')
epoch: 7
train acc: tensor(0.2311, device='cuda:0')
dev_acc: tensor(0.1990, device='cuda:0')
epoch: 8
train acc: tensor(0.2395, device='cuda:0')
dev_acc: tensor(0.2042, device='cuda:0')
epoch: 9
train acc: tensor(0.2466, device='cuda:0')
dev_acc: tensor(0.2063, device='cuda:0')
epoch: 10
train acc: tensor(0.2516, device='cuda:0')
dev_acc: tensor(0

In [ ]:
def predict(net,dataloader):
    net.eval()
    predictions = torch.tensor([]).cuda(gpu)
    with torch.no_grad():
        for i, (x,y) in enumerate(test_loader):
            x,y=x.cuda(gpu),y.cuda(gpu)
            logit = net(x)
            pred = torch.argmax(logit,dim=1)+1
            predictions=torch.cat((predictions,pred))
    return predictions.cpu().numpy()
pred = predict(net,test_loader)
print(pred)

[ 9.  3. 10. ...  6.  6.  8.]


In [ ]:
output = pd.read_csv("sample.csv")
output['Category']=pred.astype('int')
output.head()
output.to_csv("predictions.csv",index=None)

In [ ]:
data = pd.read_csv("drive/MyDrive/SML/training_set.csv",header=None)